# Draft Assistant

A code-based draft assistant (no UI).

In [5]:
# Install package
%pip install .. -q

import pandas as pd
import statisticaldrafting as sd

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load a model. 
# sd.list_sets() # Show available sets.
dm = sd.DraftModel(set="FDN", draft_mode="Premier")

In [7]:
# collection = ["Vivien Reid", "Scavenging Ooze"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
# collection = [171, 171, 225, 29, 181, 249, 239, 200]
collection = [] #["Bloodthirsty Conqueror"] * 5
pick_order = dm.get_pick_order(collection)
# pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])] # Optional filter. 
# pick_order = pick_order[pick_order["color_identity"].isin(["G"])] # Optional filter. 
pick_order
# pick_order.sort_values(by="synergy", ascending=False).head(30)

,name,rarity,color_identity,p1p1_rating,synergy,rating
85,Embercleave,special,R,5.00,0.0,5.00
162,"Liliana, Dreadhorde General",mythic,B,4.86,0.0,4.86
54,"Chandra, Flameshaper",mythic,R,4.68,0.0,4.68
63,Curator of Destinies,rare,U,4.63,0.0,4.63
35,Bloodthirsty Conqueror,mythic,B,4.63,0.0,4.63
15,"Arahbo, the First Fang",rare,W,4.40,0.0,4.40
263,Twinflame Tyrant,mythic,R,4.37,0.0,4.37
155,"Kiora, the Rising Tide",rare,U,4.30,0.0,4.30
271,Vivien Reid,mythic,G,4.27,0.0,4.27
52,Celestial Armor,rare,W,4.21,0.0,4.21


In [ ]:
# Deckbuild recommendations after draft. 
# dm.get_deck_recommendation(collection, starting_colors="")